# DAT257x: Reinforcement Learning Explained

## Lab: Dynamic Programming

### Policy Iteration

Policy Iteration calculates the optimal policy for an MDP, given its full definition.  The full definition of an MDP is the set of states, the set of available actions for each state, the set of rewards, the discount factor, and the state/reward transition function.

In [7]:
import numpy as np
import gridworld_mdp as gw   # defines the MDP for a 4x4 gridworld

**Implement the algorithm for Policy Iteration**.  Policy Iteration calculates the optimal policy for an MDP by doing repeated steps of policy evaluation and policy improvement.

Note that there is a subtle difference between the algorithm for Policy Evaluation, which assumes the policy is stochastic, and the Policy Evaluation step for the Policy Iteration algorithm, which assumes the policy is deterministic.  This means that you cannot directly call your previous code, but you can reuse large pieces of it for the Policy Evaluation step.

In [8]:
def policy_iteration(state_count, gamma, theta, get_available_actions, get_transitions):
    """
    This function computes the optimal value function and policy for the specified MDP, using the Policy Iteration algorithm.
    'state_count' is the total number of states in the MDP. States are represented as 0-relative numbers.
    
    'gamma' is the MDP discount factor for rewards.
    
    'theta' is the small number threshold to signal convergence of the value function (see Iterative Policy Evaluation algorithm).
    
    'get_available_actions' returns a list of the MDP available actions for the specified state parameter.
    
    'get_transitions' is the MDP state / reward transiton function.  It accepts two parameters, state and action, and returns
        a list of tuples, where each tuple is of the form: (next_state, reward, probabiliity).  
    """
    V = state_count*[0]                # init all state value estimates to 0
    pi = state_count*[0]
    
    # init with a policy with first available action for each state
    for s in range(state_count):
        avail_actions = get_available_actions(s)
        pi[s] = avail_actions[0]
 
    while True:
        # Policy Evaluation
        while True:
            delta = 0
            for s in range(state_count):
                v = V[s]
                tmp_value = 0
                transitions = get_transitions(state=s, action=pi[s])
                for transition in transitions:
                    next_state, reward, probability = transition
                    tmp_value += probability*(reward + gamma*V[next_state])
                V[s] = tmp_value
                delta = max(delta, abs(v - V[s]))
            if  delta < theta:
                break
        
        # Policy improvement
        print("Policy=", pi)
        stable = True
        for s in range(state_count):
            old_action = pi[s]
            q = []
            for action in get_available_actions(s):
                tmp_value = 0
                transitions = get_transitions(state=s, action=action)
                for transition in transitions:
                    next_state, reward, probability = transition
                    tmp_value += probability*(reward + gamma*V[next_state])
                q.append(tmp_value)
            a = np.argmax(q)
            pi[s] = get_available_actions(s)[a]
            if old_action != pi[s]:
                stable = False
        if stable:
            break
    
    return (V, pi)        # return both the final value function and the final policy

First, test our function using the MDP defined by gw.* functions.

In [9]:
n_states = gw.get_state_count()

# test our function
values, policy = policy_iteration(state_count=n_states, gamma=.9, theta=.001, get_available_actions=gw.get_available_actions, \
    get_transitions=gw.get_transitions)

Policy= ['up', 'up', 'up', 'up', 'up', 'up', 'up', 'up', 'up', 'up', 'up', 'up', 'up', 'up', 'up']
Policy= ['up', 'left', 'up', 'up', 'up', 'left', 'up', 'up', 'up', 'left', 'up', 'down', 'up', 'left', 'right']
Policy= ['up', 'left', 'left', 'up', 'up', 'up', 'left', 'down', 'up', 'up', 'down', 'down', 'up', 'right', 'right']
Policy= ['up', 'left', 'left', 'down', 'up', 'up', 'up', 'down', 'up', 'up', 'down', 'down', 'up', 'right', 'right']


In [10]:
a = np.append(values, 0)
np.reshape(a, (4,4))

array([[ 0.  , -1.  , -1.9 , -2.71],
       [-1.  , -1.9 , -2.71, -1.9 ],
       [-1.9 , -2.71, -1.9 , -1.  ],
       [-2.71, -1.9 , -1.  ,  0.  ]])

In [11]:
a = np.append(policy, policy[0])
np.reshape(a, (4,4))

array([['up', 'left', 'left', 'down'],
       ['up', 'up', 'up', 'down'],
       ['up', 'up', 'down', 'down'],
       ['up', 'right', 'right', 'up']], dtype='<U5')